## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [26]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

--2020-12-20 16:43:01--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6023:18::a27d:4312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2020-12-20 16:43:01--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd343bb9c0e39b1bdf5360c6666.dl.dropboxusercontent.com/cd/0/inline/BFabTcvTHWiFmUThHVE8eLGth7YL_7wGt-khLh7zfIkIRUSkLz3q81hHQ2Towm7QcmF0Vxy90UyitlNqjq5tlBnDVE_mtuDClJK57CfFmaz2U5yPooyjuT06IVaaqic709w/file# [following]
--2020-12-20 16:43:02--  https://ucd343bb9c0e39b1bdf5360c6666.dl.dropboxusercontent.com/cd/0/inline/BFabTcvTHWiFmUThHVE8eLGth7YL_7wGt-khLh7zfIkIRUSkLz3q81hHQ2Towm7QcmF0Vxy90UyitlNqjq5tlBnDVE_mtuDClJK

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [27]:
# exercise1
import pandas as pd
import numpy as np

file_name = "data_000637.txt"
N = 2000
data=pd.read_csv(file_name)
df = pd.DataFrame(data)
df.head(N)


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1995,1,0,62,3869200187,1979,16
1996,1,0,61,3869200187,1984,9
1997,1,0,60,3869200187,2067,25
1998,1,0,59,3869200187,2071,13


In [28]:
#exercise 2
x = df['BX_COUNTER'].max()
#c = data.BX_COUNTER.max()
co = x * 20
print(co)

71260


In [29]:
#exercise3
time = data.loc[0, 'TDC_MEAS'] * 21/30 + data.loc[0,'BX_COUNTER'] * 21 + data.loc[0,'ORBIT_CNT']*co
time_N = data.loc[(N-1), 'TDC_MEAS']* 23/30 + data.loc[(N-1),'BX_COUNTER']*22 + data.loc[(N-1),'ORBIT_CNT']*co

print("Duration:", (time_N - time)*10**(-9))

Duration: 0.00142096653125


In [30]:
#exercise 4
data['timens'] = data['TDC_MEAS'] * 22/30 + data['BX_COUNTER'] * 29 + data['ORBIT_CNT'] * co  - time
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,timens
0,1,0,123,3869200167,2374,26,1.899288e+04
1,1,0,124,3869200167,2374,27,1.899362e+04
2,1,0,63,3869200167,2553,28,2.418534e+04
3,1,0,64,3869200167,2558,19,2.432375e+04
4,1,0,64,3869200167,2760,25,3.018616e+04
...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,7.841173e+08
1310716,1,1,4,3869211171,763,11,7.841173e+08
1310717,1,0,64,3869211171,764,0,7.841173e+08
1310718,1,0,139,3869211171,769,0,7.841175e+08


In [31]:
#exercise 5

df['HEAD'] = np.random.randint(0,2, size = len(df))
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,timens
0,1,0,123,3869200167,2374,26,1.899288e+04
1,0,0,124,3869200167,2374,27,1.899362e+04
2,1,0,63,3869200167,2553,28,2.418534e+04
3,1,0,64,3869200167,2558,19,2.432375e+04
4,0,0,64,3869200167,2760,25,3.018616e+04
...,...,...,...,...,...,...,...
1310715,0,0,62,3869211171,762,14,7.841173e+08
1310716,0,1,4,3869211171,763,11,7.841173e+08
1310717,0,0,64,3869211171,764,0,7.841173e+08
1310718,0,0,139,3869211171,769,0,7.841175e+08


In [32]:
#exercise 6
df_1 = df.loc[df['HEAD'] == 1]
df_1

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,timens
0,1,0,123,3869200167,2374,26,1.899288e+04
2,1,0,63,3869200167,2553,28,2.418534e+04
3,1,0,64,3869200167,2558,19,2.432375e+04
7,1,0,139,3869200167,2776,0,3.063181e+04
9,1,0,60,3869200167,2788,7,3.098494e+04
...,...,...,...,...,...,...,...
1310708,1,0,124,3869211171,199,9,7.841009e+08
1310709,1,1,38,3869211171,417,8,7.841073e+08
1310712,1,1,35,3869211171,429,4,7.841076e+08
1310714,1,0,60,3869211171,762,2,7.841173e+08


In [41]:
#exercise 8
var = data.groupby('TDC_CHANNEL').count().sort_values('HEAD', ascending = False)
print(var.head(5))

               HEAD    FPGA  ORBIT_CNT  BX_COUNTER  TDC_MEAS  timens
TDC_CHANNEL                                                         
139          108059  108059     108059      108059    108059  108059
64            66020   66020      66020       66020     66020   66020
63            64642   64642      64642       64642     64642   64642
61            49479   49479      49479       49479     49479   49479
62            49349   49349      49349       49349     49349   49349


In [36]:
#exercise 9

df_new = df.groupby('ORBIT_CNT')
N = df_new.count()
print("count of unique orbits:", N)

df_new2 = df.loc[df['TDC_CHANNEL'] == 139] 
df_new2 = df_new2.groupby('ORBIT_CNT')
var = df_new2.count()
print(var)

count of unique orbits:             HEAD  FPGA  TDC_CHANNEL  BX_COUNTER  TDC_MEAS  timens
ORBIT_CNT                                                        
3869200167    43    43           43          43        43      43
3869200168    85    85           85          85        85      85
3869200169   127   127          127         127       127     127
3869200170    98    98           98          98        98      98
3869200171   109   109          109         109       109     109
...          ...   ...          ...         ...       ...     ...
3869211167   208   208          208         208       208     208
3869211168   109   109          109         109       109     109
3869211169   191   191          191         191       191     191
3869211170   137   137          137         137       137     137
3869211171    22    22           22          22        22      22

[11001 rows x 6 columns]
            HEAD  FPGA  TDC_CHANNEL  BX_COUNTER  TDC_MEAS  timens
ORBIT_CNT                 